In [2]:
import _pickle as pickle
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import copy
import glob
import gc

In [3]:
from sklearn import metrics

In [4]:
pd.set_option('max_rows',400)

In [5]:
# f = open('/data/Bodenreider_UMLS_DL/Interns/Bernal/uva_testing_data.csv','r')

# f.readline()

# testing_data = []

# for line in tqdm(f,total=173000000):
#     line = line.strip()
#     _,_,_,_,l,_,h,_,t = line.split('\t')
    
#     testing_data.append((h,t,l))

In [ ]:
# testing_data_df = pd.DataFrame(testing_data)

In [ ]:
# validation_set = []
# testing_set = []

# for i,g in testing_data_df.groupby(2):
    
#     perm = g.sample(len(g), random_state=np.random.RandomState(42))
    
#     val = perm[:int(len(g)*0.01)]
#     test = perm[int(len(g)*0.01):]
    
#     validation_set.append(val)
#     testing_set.append(test)

In [ ]:
# validation_set = pd.concat(validation_set)
# testing_set = pd.concat(testing_set)

In [ ]:
# len(validation_set)

In [ ]:
# validation_set.groupby(2).count()

In [ ]:
# validation_set_tuples = [(h,t,l) for h,t,l in zip(validation_set[0],validation_set[1],validation_set[2])]

In [ ]:
# pickle.dump(validation_set_tuples, open('/data/Bodenreider_UMLS_DL/Interns/Bernal/uva_test_subset.p','wb'))

In [95]:
validation_set_tuples = pickle.load(open('/data/Bodenreider_UMLS_DL/Interns/Bernal/uva_test_subset.p','rb'))
pred_filename = '/data/Bodenreider_UMLS_DL/Interns/Bernal/ubert_mlm_uva_test_290020_2_predictions/umls2020_ubert_mlm_uva_test_290020_2_predictions_{}.p'

In [113]:
validation_set_tuples = pickle.load(open('/data/Bodenreider_UMLS_DL/Interns/Bernal/uva_test_subset.p','rb'))
pred_filename = '/data/Bodenreider_UMLS_DL/Interns/Bernal/sapbert_ubert_uva_test_1392096_predictions/umls2020_sapbert_ubert_uva_test_1392096_predictions_{}.p'

In [133]:
validation_set_tuples = pickle.load(open('/data/Bodenreider_UMLS_DL/Interns/Bernal/umls2020AB_sapbert_edges.p','rb'))
pred_filename = '/data/Bodenreider_UMLS_DL/Interns/Bernal/ubert_mlm_290020_2_predictions/umls2020_ubert_mlm_290020_2_predictions_{}.p'

In [186]:
validation_set_tuples = pickle.load(open('/data/Bodenreider_UMLS_DL/Interns/Bernal/umls2020AB_sapbert_edges.p','rb'))
pred_filename = '/data/Bodenreider_UMLS_DL/Interns/Bernal/sapbert_ubert_1392096_predictions/umls2020_sapbert_ubert_1392096_predictions_{}.p'

In [47]:
validation_set_tuples = pickle.load(open('/data/Bodenreider_UMLS_DL/Interns/Bernal/umls2020AB_sapbert_edges_subset.p','rb'))
pred_filename = '/data/Bodenreider_UMLS_DL/Interns/Bernal/sapbert_ubert_rw_uva_subset_1392096_predictions/umls2020_sapbert_ubert_rw_uva_subset_1392096_predictions_{}.p'

In [27]:
validation_set_tuples = pickle.load(open('/data/Bodenreider_UMLS_DL/Interns/Bernal/umls2020AB_sapbert_edges_subset.p','rb'))
pred_filename = '/data/Bodenreider_UMLS_DL/Interns/Bernal/ubert_mlm_rw_uva_subset_290020_2_predictions/umls2020_ubert_mlm_rw_uva_subset_290020_2_predictions_{}.p'

In [29]:
validation_set_tuples = pickle.load(open('/data/Bodenreider_UMLS_DL/Interns/Bernal/rw_uva/data/ubert_2020AB_dev_set.p','rb'))
pred_filename = '/data/Bodenreider_UMLS_DL/Interns/Bernal/ubert_mlm_rw_uva_2020AB_dev_set_290020_2_predictions/umls2020_ubert_mlm_rw_uva_2020AB_dev_set_290020_2_predictions_{}.p'

In [6]:
validation_set_tuples = pickle.load(open('/data/Bodenreider_UMLS_DL/Interns/Bernal/rw_uva/data/ubert_2020AB_dev_set.p','rb'))
pred_filename = '/data/Bodenreider_UMLS_DL/Interns/Bernal/sapbert_ubert_rw_uva_2020AB_dev_set_1392096_predictions/umls2020_sapbert_ubert_rw_uva_2020AB_dev_set_1392096_predictions_{}.p'

In [30]:
preds = []
i=0

while os.path.exists(pred_filename.format(i)):
    f = open(pred_filename.format(i),'rb')
    preds.append(pickle.load(f))
    i+=1
    
preds = np.vstack(preds)

In [31]:
len(preds)/2

146255.0

In [32]:
len(validation_set_tuples)

146255

In [33]:
edges_to_test = validation_set_tuples[:int(len(preds)/2)]

In [34]:
len(validation_set_tuples)

146255

In [35]:
forward_preds = []
backward_preds = []

for i,p in enumerate(preds):
    if i % 2 == 0:
        forward_preds.append(p)
    else:
        backward_preds.append(p)

In [36]:
edges_to_test_df = pd.DataFrame(edges_to_test)

In [37]:
len(edges_to_test_df)

146255

In [38]:
edges_to_test_df['forward_pred'] = np.argmax(np.vstack(forward_preds),axis=1)
edges_to_test_df['backward_pred'] = np.argmax(np.vstack(backward_preds),axis=1)

edges_to_test_df['forward_pred'] = [e for e in edges_to_test_df['forward_pred']]
edges_to_test_df['backward_pred'] = [e for e in edges_to_test_df['backward_pred']]

In [39]:
edges_to_test_df['f_probs'] = forward_preds
edges_to_test_df['b_probs'] = backward_preds

In [40]:
pd.set_option('max_colwidth',100)

In [41]:
#Reverse labels
edges_to_test_df[2] = [not(int(e)) for e in edges_to_test_df[2]]

In [42]:
edges_to_test_df

,0,1,2,forward_pred,backward_pred,f_probs,b_probs
0,Acacia sp. CJ7,Acacia sp. CJ28,True,1,1,"[-4.728855, 5.2853227]","[-4.2643113, 5.942383]"
1,Cat recombinant (rFel d) 7,"IL-6 PE toxin, chimeric",True,1,1,"[-2.8679383, 11.791564]","[-2.8665717, 11.789539]"
2,Staphylococcus virus 2638A,Staphylococcus sp. K18D3,True,1,1,"[-2.8768692, 11.8052435]","[-5.0928936, 3.159361]"
3,BENZALKONIUM CHLORIDE 0.1 g in 100 mL TOPICAL CLOTH [Baby Wipes],BENZALKONIUM CHLORIDE 0.143 g TOPICAL CLOTH [PRE-MOISTENED BATHING CLOTHS],False,0,1,"[8.135373, -6.371258]","[-0.45245203, 0.7152366]"
4,polyethylene glycol 3350 Powder for Oral Solution [Gavilax],"POLYETHYLENE GLYCOL 3350 17 g in 17 g ORAL POWDER, FOR SOLUTION [CLEARLAX]",True,1,1,"[-2.8173544, 11.725847]","[-2.8686154, 11.792815]"
...,...,...,...,...,...,...,...
146250,cysteine-rich protein 1 (intestinal) pseudogene 4,complement C1q binding protein pseudogene 3,True,1,1,"[-2.8775003, 11.806282]","[-2.8794227, 11.809525]"
146251,Muscodor sp. EXH1_22,Muscidae sp. BIOUG17011-F04,True,1,1,"[-2.8855402, 11.820677]","[-2.8864388, 11.822835]"
146252,Variovorax sp. mfec-39,Variovorax sp. JCM 16519,True,0,0,"[7.696317, -5.2156262]","[2.6450667, -2.9767125]"
146253,cefotaxime 1000 MG Injection,Claforan (cefotaxime sodium) 10 GM per 50 ML Injectable Solution,True,1,1,"[-2.6332421, 3.027621]","[-3.8293495, 4.0340133]"


In [43]:
edges_to_test_df[(edges_to_test_df[2] == 0)][:100]

,0,1,2,forward_pred,backward_pred,f_probs,b_probs
3,BENZALKONIUM CHLORIDE 0.1 g in 100 mL TOPICAL CLOTH [Baby Wipes],BENZALKONIUM CHLORIDE 0.143 g TOPICAL CLOTH [PRE-MOISTENED BATHING CLOTHS],False,0,1,"[8.135373, -6.371258]","[-0.45245203, 0.7152366]"
8,"wingless-type MMTV integration site family, member 11",WNT11 gene,False,0,0,"[1.6890092, -1.6687374]","[6.8737726, -7.958695]"
93,BENZALKONIUM CHLORIDE 0.1 g in 100 g TOPICAL LIQUID [Pullio Hand Sanitizing Wipes Peppermint],BENZALKONIUM CHLORIDE 0.00097 g in 1 mL TOPICAL CLOTH [Healthy Hands Sanitizing Hand Wipes],False,0,0,"[4.253632, -4.170172]","[1.878714, -1.8187454]"
96,HSCB MITOCHONDRIAL IRON-SULFUR CLUSTER COCHAPERONE,DNAJC20,False,0,0,"[8.139614, -5.074443]","[6.3092546, -4.8570533]"
110,Leader Witch Hazel 86% Topical Solution,WITCH HAZEL 86 mg in 100 mL TOPICAL LIQUID,False,0,0,"[6.2159534, -4.9509096]","[7.331353, -7.5063834]"
172,BENZALKONIUM CHLORIDE 1 mg in 1 g TOPICAL CLOTH [GSD Good Hygiene Defense Wet Wipes],"benzalkonium chloride 0.1 %-1 3/4"" X 4"" TOPICAL ADHESIVE PATCH, MEDICATED",False,0,0,"[8.159208, -5.3545113]","[8.040807, -5.5139265]"
187,amitriptyline hydrochloride 12.5 MG / chlordiazePOXIDE 5 MG Oral Tablet,"Amitriptyline-Chlordiazepoxide, 12.5 mg-5 mg oral tablet",False,0,0,"[8.144306, -6.351211]","[6.305233, -4.8541255]"
193,Esophageal; Proximal,Structure of upper third of esophagus,False,0,0,"[6.6321816, -4.9455905]","[8.045862, -6.525316]"
207,BENZALKONIUM CHLORIDE 0.1 mg in 100 mg TOPICAL SOLUTION [Sanitizer Wipes],BENZALKONIUM CHLORIDE 0.00097 g in 1 mL TOPICAL CLOTH [SCRUBS Hand Sanitizer Wipe],False,0,1,"[8.2583275, -6.1153774]","[-1.6490378, 1.9377663]"
211,Enterocloster lavalensis,Clostridium lavalense,False,1,0,"[-0.3304227, 0.61727655]","[1.0491049, -0.84913665]"


In [44]:
edges_to_test_df[(edges_to_test_df[2] != edges_to_test_df['forward_pred']) & (edges_to_test_df[2] == 1)][:10]

,0,1,2,forward_pred,backward_pred,f_probs,b_probs
6,ECPAS,adenylyl cyclase-activating protease,True,0,0,"[7.868511, -6.8112025]","[7.5788903, -7.252938]"
13,Alibertia <eudicots>,Alienobostra,True,0,0,"[3.3828893, -3.63391]","[2.4276865, -2.6182346]"
15,Mortierella sp. SM14-21-28-2,Mortierella sp. 1 GB-2015,True,0,0,"[0.63189304, -0.41665158]","[0.28481916, -0.038662903]"
30,Boana caingua,Javania cailleti,True,0,0,"[3.1918645, -3.5440543]","[4.21998, -4.120028]"
32,"PabB protein, E coli","pdxB protein, E coli",True,0,0,"[6.2928457, -4.83548]","[5.724019, -4.6263885]"
43,H2AZP2,"H2A HISTONE FAMILY, MEMBER Y2",True,0,0,"[8.356167, -5.6786466]","[6.962629, -7.8722744]"
45,C20orf50,C17orf70,True,0,0,"[8.171422, -6.358363]","[6.560381, -4.925057]"
46,fibrinogen concentrate (human) 1 MG Injection [Fibryga],FIBRINOGEN (HUMAN) 900-1300MG/VIL INJ,True,0,1,"[1.1767141, -1.0057602]","[-4.8161244, 5.169194]"
47,Rasahus sulcicollis,Musgraveia sulciventris,True,0,0,"[1.3939109, -1.3172381]","[0.74702865, -0.5196144]"
49,C13orf8,C1orf12,True,0,1,"[2.5480666, -2.814645]","[-2.279213, 2.5230317]"


In [45]:
edges_to_test_df.groupby(2).count()

,0,1,forward_pred,backward_pred,f_probs,b_probs
2,,,,,,
False,8239,8239,8239,8239,8239,8239
True,138016,138016,138016,138016,138016,138016


In [46]:
edges_to_test_df.groupby('forward_pred').count()

,0,1,2,backward_pred,f_probs,b_probs
forward_pred,,,,,,
0,31787,31787,31787,31787,31787,31787
1,114468,114468,114468,114468,114468,114468


In [47]:
edges_to_test_df.groupby('backward_pred').count()

,0,1,2,forward_pred,f_probs,b_probs
backward_pred,,,,,,
0,35036,35036,35036,35036,35036,35036
1,111219,111219,111219,111219,111219,111219


In [48]:
labels = [int(e) for e in edges_to_test_df[2]]

In [49]:
f_p, f_r, forward_ubert_f1, f_s = metrics.precision_recall_fscore_support(labels, edges_to_test_df['forward_pred'], pos_label=0, average='binary')
b_p, b_r, backward_ubert_f1, b_s = metrics.precision_recall_fscore_support(labels, edges_to_test_df['backward_pred'], pos_label=0, average='binary')

In [50]:
forward_ubert_f1, f_p, f_r

(0.34987258282116623, 0.22027873029854972, 0.8498604199538778)

In [51]:
backward_ubert_f1, b_p, b_r

(0.3360831889081456, 0.20755794040415573, 0.8826313873042845)

In [138]:
f = open('/data/Bodenreider_UMLS_DL/thilini/EXPERIMENTS/datasets/2020AA-ACTIVE_ALL_MODEL_GENTEST_DS_TEST_DS_SHUF_dedup.RRF','r')

test_set = []

for i, line in tqdm(enumerate(f),total=173000000):
    line = line.strip()
    
#     if i > 100000:
#         break
    
    line_list = line.split('|')
    test_set.append(line_list)

 99%|███████████████████████████████████████████████████████████████████████████████▌| 171991918/173000000 [05:55<00:02, 484468.27it/s]


In [87]:
f = open('/data/Bodenreider_UMLS_DL/thilini/EXPERIMENTS/6_SAPBERT_SP/test/correct_metrics_predictions_checkpoint-1392096.txt','r')
f = open('/data/Bodenreider_UMLS_DL/thilini/EXPERIMENTS/1_UMLS_ONLY/testing/correct_metrics_predictions_checkpoint-290020_2.txt','r')

preds = []


for i, line in tqdm(enumerate(f),total=173000000):
    line = line.strip()
    
#     if i > 100000:
#         break
        
    preds.append([float(f) for f in line.split()])

 99%|███████████████████████████████████████████████████████████████████████████████▌| 171991918/173000000 [17:09<00:06, 167078.73it/s]


In [89]:
f = open('/data/Bodenreider_UMLS_DL/thilini/EXPERIMENTS/6_SAPBERT_SP/test/correct_metrics_labels_checkpoint-986068.txt','r')
# f = open('/data/Bodenreider_UMLS_DL/thilini/EXPERIMENTS/1_UMLS_ONLY/testing/correct_metrics_labels_checkpoint-435030_2.txt','r')

true = []

for i, line in tqdm(enumerate(f),total=173000000):
    line = line.strip()
    
#     if i > 100000:
#         break
    
    true.append(line)

 99%|███████████████████████████████████████████████████████████████████████████████▌| 171991918/173000000 [03:51<00:01, 742297.51it/s]


In [183]:
aui_dict = pickle.load(open('/data/Bodenreider_UMLS_DL/thilini/EXPERIMENTS/aui_vec/aui_vec.pkl','rb'))

In [90]:
true = [int(l) for l in true]

In [154]:
#Alternative
test_set_df = pd.DataFrame(test_set)
l_ = [int(e) for e in test_set_df[3]]
l_ = np.array(l_)
ls = (l_ + 1) % 2
true = ls

In [92]:
preds_ = np.argmax(preds, axis=1)

In [93]:
sum(true),sum(preds_),len(true), len(preds_)

(166410710, 165921774, 171991918, 171991918)

In [94]:
p, r, f1, _ = metrics.precision_recall_fscore_support(true, preds_, pos_label=0, average='binary')

In [173]:
p,r,f1

(0.8950508812636843, 0.9757724492618802, 0.9336701936246622)

In [174]:
test_set_df['labels'] = true

In [175]:
test_set_df['preds'] = preds_

In [176]:
test_set_df['probs'] = preds

In [118]:
aui_info = []

with open('/data/Bodenreider_UMLS_DL/UMLS_VERSIONS/2020AA-ACTIVE/META/MRCONSO.RRF','r') as fp:
    
    for line in fp.readlines():
        line = line.split('|')
        cui = line[0]
        aui = line[7]
        string = line[-5]
        
        aui_info.append({'AUI':aui, 'CUI':cui, 'STR':string})
        
cui2sg = {}

with open('/data/Bodenreider_UMLS_DL/UMLS_VERSIONS/2020AA-ACTIVE/META/MRSTY.RRF','r') as fp:
    
    for line in fp.readlines():
        line = line.split('|')
        cui = line[0]
        sg = line[3]
        cui2sg[cui] = sg
        
cui2aui = {}
aui2cui = {}
aui2str = {}
aui2sg = {}

cui_sg = []
cui_aui = []

for tup in tqdm(aui_info):
    aui = tup['AUI']
    
    cui = tup['CUI']
    string = tup['STR']
    sg = cui2sg[cui]

    auis = cui2aui.get(cui, [])
    auis.append(aui)
    cui2aui[cui] = auis

    aui2cui[aui] = cui
    aui2str[aui] = string
    aui2sg[aui] = sg

    cui_sg.append((cui, sg))
    cui_aui.append((cui, aui))
        
semgroups = pd.read_csv('SemGroups.txt',sep='|',header=None)

semtype2sg = {}

for i, row in semgroups.iterrows():
    
    st = row[3]
    sg = row[1]
    
    semtype2sg[st] = sg

100%|██████████████████████████████████████████████████████████████████████████████████| 14770662/14770662 [00:49<00:00, 297781.09it/s]


In [131]:
aui2str['A27392030'], aui2str['A30080569']

('OXYGEN 990 mL in 1 L RESPIRATORY (INHALATION) GAS',
 'OXYGEN 99 L in 100 L RESPIRATORY (INHALATION) GAS')

In [177]:
subset = []

for i,g in test_set_df.groupby('labels'):
    
    ss = g[:int(len(g)*0.0001)]
    subset.append(ss)
    
subset = pd.concat(subset)

In [178]:
subset.groupby('labels').count()

,0,1,2,3,preds,probs
labels,,,,,,
0,558,558,558,558,558,558
1,16641,16641,16641,16641,16641,16641


In [179]:
subset['aui1'] = [aui2str[a] for a in subset[1]]
subset['aui2'] = [aui2str[a] for a in subset[2]]

In [180]:
subset_tuples = []

for i, row in subset.iterrows():
    
    subset_tuples.append((row['aui1'],row['aui2'],row['labels']))

In [60]:
# pickle.dump(subset_tuples, open('/data/Bodenreider_UMLS_DL/Interns/Bernal/uva_test_subset.p','wb'))

In [181]:
len(subset_tuples)

17199

In [182]:
subset

,0,1,2,3,labels,preds,probs,aui1,aui2
10,,A30080569,A27392030,1,0,0,"[9.83976, -5.19342]",OXYGEN 99 L in 100 L RESPIRATORY (INHALATION) GAS,OXYGEN 990 mL in 1 L RESPIRATORY (INHALATION) GAS
23,,A18609178,A11919293,1,0,0,"[10.09789, -4.9064]",chlorambucil,"para-N,N-di(b-chloroethyl)aminophenylbutyric acid"
89,,A20995170,A30930630,1,0,0,"[5.99245, -5.43056]",Aspirin 325 mg in 325 mg ORAL TABLET [Value PharmaPain Reliever Extra Strength],"ASPIRIN 325 mg ORAL TABLET, COATED [aspirin pain relief]"
99,,A12107333,A18464903,1,0,1,"[0.00327, 0.02176]","SPINOCEREBELLAR ATAXIA, X-LINKED 3","Spinocerebellar ataxia, X-linked, 3"
119,,A28944944,A31462601,1,0,0,"[9.88996, -5.15059]",toldimfos sodium 200 MG/ML Injectable Solution [Phos-Aid],TOLDIMFOS SODIUM 200 mg in 1 mL INTRAMUSCULAR INJECTION [Phos-Aid]
...,...,...,...,...,...,...,...,...,...
17208,,A19314985,A20897915,0,1,1,"[-2.89513, 11.96102]",eprinomectin,eprinomectin 50 MG/ML
17209,,A26465514,A28763437,0,1,1,"[-2.89377, 11.9591]",Regurgitation,mitral regurgitation due to acute myocardial infarction (diagnosis)
17210,,A2979722,A15489415,0,1,1,"[-2.91038, 11.98175]",Reticular dysgenesia,Nervous system of pectoral girdle
17211,,A11713707,A0062665,0,1,1,"[-2.90471, 11.97332]",COQ9 gene,Genes


In [184]:
aui_dict['A30080569']

[3208, 2883, 54, 194, 549, 54, 1351, 12, 3617, 13, 1482]